# Selection Process

This is a demonstration of a chain of widgets that can be used in the data plotting process.  Several of these widgets are dependant on the selected values of the previous widgets.

In [ ]:
import ipywidgets
import IPython.display
import iris

import glob
import numpy as np
import iris.quickplot as iplt
import matplotlib.pyplot as plt

# 1. Choose and load your cube

Create cube picker consiting of a text input box for the file path and a selection widget for the cube selector.

In [ ]:
path = ipywidgets.Text(
    description='Filepath',
    value='/tmp',
    width='100%')

IPython.display.display(path)

In [ ]:
options = glob.glob('{}/*'.format(path.value)) 

files = ipywidgets.Select(
    description='Cube(list)',
    options=options,
    width='100%')
IPython.display.display(files)

Make a variable to store and load the cube or cubes in the file you have selected.

In [ ]:
cubes = files.value

cubelist = iris.load_raw(cubes)

print cubelist        

Create a selection pane for choosing which cube to plot.

In [ ]:
to_plot = [cube.standard_name for cube in cubelist]

plots = ipywidgets.Multiple(
description='Choose cube(s)',
options=to_plot)

IPython.display.display(plots)

# 2. Choose your plot options

Create plot-type picker

In [ ]:
plot_type_dict = {'contour': iplt.contour, 'contourf': iplt.contourf, 'pcolor': iplt.pcolor, 'outline': iplt.outline,
                  'pcolormesh': iplt.pcolormesh, 'plot': iplt.plot, 'points': iplt.points}

plot_types = plot_type_dict.keys()
plot_types.sort()

type = ipywidgets.Dropdown(
    options=plot_types,
    value='contour',
    description='Plot-type:')

IPython.display.display(type)

Create axis coordinate pickers for x and y axis.

In [ ]:
base_cube = iris.load_cube(files.value, plots.value[0])
coordinates = [(coord.name()) for coord in base_cube.coords()]

for i in range(len(plots.value)):
    coordinates.append(plots.value[i])

dim_x = ipywidgets.RadioButtons(
    description='Dimension for x:',
    options=coordinates)

IPython.display.display(dim_x)

In [ ]:
if dim_x.value in coordinates:
    coordinates.remove(dim_x.value)

dim_y = ipywidgets.RadioButtons(
    description='Dimension for y:',
    options=coordinates)

IPython.display.display(dim_y)

# 3. Choose your formatting options

Construct a set of widgets to present options for plot customization.  Some of the options are only applicable to certain types of plots, so these widgets are dependant on the value of the plot-type widget.

In [ ]:
# All:
colors = ipywidgets.Dropdown(
    options=['None', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='colors')
cmap = ipywidgets.Dropdown(
    options=['None', 'viridis', 'inferno', 'plasma', 'magma', 'jet', 'summer', 'autumn'],
    value='None',
    description='cmap')
norm = ipywidgets.Dropdown(
    options=['None', 'Autoscale(A)', 'Autoscale_None(A)', 'scaled()'],
    value='None',
    description='norm')
levels = ipywidgets.Text(
    description='levels',
    value='np.linspace(np.min(cube.data), np.max(cube.data), 10)')
origin = ipywidgets.Dropdown(
    options=['None', 'upper', 'lower', 'image'],
    value='None',
    description='origin')
extend = ipywidgets.Dropdown(
    options=['neither', 'both', 'min', 'max'],
    value='both',
    description='extend')
# Contour-only:
linestyles = ipywidgets.Dropdown(
    options=['None', 'solid', 'dashed', 'dashdot', 'dotted'],
    value='None',
    description='linestyle')
# Plot only:
label = ipywidgets.Text(
    description='label',
    value='Line Plot')
# Plot and Points only:
color = ipywidgets.Dropdown(
    options=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='color')
marker = ipywidgets.Dropdown(
    options=['-', '--', '-.', ':', '.', ',', 'o', 'v', '^', '<', '>', '*', 'x'],
    value='-',
    description='marker')
    
formatting = [colors, cmap, norm, levels, origin, extend]
    
if type.value in ['contourf', 'outline', 'pcolor', 'pcolormesh']:
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'contour':
    formatting.append(linestyles)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'plot':
    formatting.append(label)
    formatting.append(color)
    formatting.append(marker)
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
elif type.value == 'points':
    formatting.remove(colors)
    formatting.append(color)
    formatting.append(marker)    
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])
    